In [18]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import gzip
import numpy as np
import tensorflow as tf
from typing import Tuple

Now that we understand how to get the data and model, we're ready to train the neural network. Let's first encapsulate the data-fetching steps and model-building steps described earlier in functions called `get_data` and `get_model`, respectively:

In [19]:
def read_images(path: str, image_size: int, num_items: int) -> np.ndarray:
  f = gzip.open(path,'r')
  buf = f.read(image_size * image_size * num_items)
  data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
  data = data.reshape(num_items, image_size, image_size)
  return data


def read_labels(path: str, num_items: int) -> np.ndarray:
  f = gzip.open(path,'r')
  f.read(8)
  buf = f.read(num_items)
  data = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
  data = data.reshape(num_items)
  return data


def get_data(batch_size: int) -> Tuple[tf.data.Dataset, tf.data.Dataset]:
  image_size = 28
  num_train = 60000
  num_test = 10000

  training_images = read_images('data/FashionMNIST/raw/train-images-idx3-ubyte.gz', image_size, num_train)
  test_images = read_images('data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz', image_size, num_test)
  training_labels = read_labels('data/FashionMNIST/raw/train-labels-idx1-ubyte.gz', num_train)
  test_labels = read_labels('data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz', num_test)

  # (training_images, training_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

  train_dataset = tf.data.Dataset.from_tensor_slices((training_images, training_labels))
  test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

  train_dataset = train_dataset.map(lambda image, label: (float(image) / 255.0, label))
  test_dataset = test_dataset.map(lambda image, label: (float(image) / 255.0, label))

  train_dataset = train_dataset.batch(batch_size).shuffle(500)
  test_dataset = test_dataset.batch(batch_size).shuffle(500)

  return (train_dataset, test_dataset)


def get_model() -> tf.keras.Sequential:
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10)
  ])
  return model

In order to understand what happens during training, we need to add a little more detail to our neural network visualization:

![Basic neural network with details](notebooks/images/2-basic-nn-with-details.png)

Notice that we've added weights $W$ to the connections between layers, and bias $b$ as input to `Dense` layers &mdash; $W$ and $b$ are the neural network's parameters. Our goal when training our network (also known as fitting) is to find the parameters $W$ and $b$ that minimize the differences between the actual and predicted labels for our data. In order to do this, we add a loss function at the end of our network, which compares the actual and predicted values, returning a large number if they're different and a smaller number if they're similar. There are many ways to write this function, and in this sample we'll use the [`SparseCategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy), which is provided to us by Keras.

When we introduce a `Dense` layer, a linear operation involving the input data and parameters $W$ and $b$ is performed. For example, the top node of the first linear layer performs the following calculation:

$$
z^1_1 = w^1_{1,1} x_1 + ... + w^1_{1,784} x_{784} + b^1_1
$$

If we specify a ReLU activation function, the output of the linear operation is then passed as input to a ReLU function:

$$
a^1_1 = ReLU(z^1_1)
$$

Mathematically speaking, we can now think of our neural network as a function $\ell$ that takes as input the data $X$, expected labels $y$, and parameters $W$ and $b$, then performs a sequence of operations on that data, and returns a loss. 

$$
\mathrm{loss} = \ell(X, y, W, b)
$$

Our goal is to find the parameters $W$ and $b$ that lead to the lowest possible loss. (We can't change our data $X$ or the corresponding labels $y$ &mdash; they're fixed &mdash; but we can adjust $W$ and $b$.) It turns out that problems of this kind fall in the well-studied mathematical area of optimization. The simplest minimization algorithm is gradient descent, and in this sample we use a variation known as Stochastic Gradient Descent or [`SGD`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD). 

We're now ready to "compile" the model &mdash; this is where we tell it that we want to use the `SGD` optimizer and the `SparseCategoricalCrossentropy` loss function. We also tell the model that we want it to report on the accuracy during training. 

In [20]:
learning_rate = 0.1
batch_size = 64

(train_dataset, test_dataset) = get_data(batch_size)

model = get_model()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['accuracy']
model.compile(optimizer, loss_fn, metrics)

A few details from the code above deserve a quick explanation. 

Notice that we pass `from_logits=True` to the loss function. Remember that the predicted value $y'$ is a vector with 10 values, but the label $y$ is an unsigned integer between 0 and 9. Before we can compare the two, we need to get the index of $y'$ with the greatest value (the `argmax`), which is a value from 0 to 9. Passing `from_logits=True` as a parameter does just that. Only then can the loss function compare $y$ and $y'$.

Notice also that we pass a `learning_rate` to the `SGD` optimizer. The learning rate is a parameter needed in the gradient descent algorithm. We could have left it at the default, which is 0.01. But it's important to know how to specify it because different learning rates can lead to very different prediction accuracies.

Finally, notice that we specified a `batch_size`, which we used in the construction of the `Dataset`, as we saw earlier. This is important during training, because it tells the model that we want to train on 64 images at a time. You might be wondering why 64? Why not train on a single image at a time? Or all 60,000 images at once? Doing a complete training step for each individual image would be inefficient because we would have to perform all the calculations 60,000 times in order to account for every input image. If we included all the input images in $X$, we'd need a lot of memory, and we'd spend a lot of time computing each training step. So we settle for a size in between, called the "mini-batch" size. 

Now that we've configured our model with the parameters we need for training, we can call `fit` to train the model. We specify the number of epochs as 2, which means that we want to iterate over the complete set of 60,000 training images twice while training the neural network. We use just two epochs in this sample because we don't want to spend too much time training the network in a demo. In a real project, you will want to increase the number of epochs, which will lead to better accuracy.

In [21]:
epochs = 2
print('\nFitting:')
model.fit(train_dataset, epochs=epochs)


Fitting:
Epoch 1/2
938/938 [==============================] - 9s 8ms/step - loss: 0.5541 - accuracy: 0.8028
Epoch 2/2
938/938 [==============================] - 8s 8ms/step - loss: 0.3964 - accuracy: 0.8554


Training has found values for the parameters $W$ and $b$ such that, when we provide an image as input, we'll get a reasonable prediction as output. Our model is now ready to be tested. Remember that when we loaded the data, we obtained two datasets, one with training data and another with test data. It's time to use the test dataset.

In [22]:
print('\nEvaluating:')
(test_loss, test_accuracy) = model.evaluate(test_dataset)
print(f'\nTest accuracy: {test_accuracy * 100:>0.1f}%, test loss: {test_loss:>8f}')


Evaluating:
157/157 [==============================] - 1s 5ms/step - loss: 0.3917 - accuracy: 0.8581

Test accuracy: 85.8%, test loss: 0.391694


Ideally, the test loss and accuracy will be pretty similar to the training loss and accuracy printed earlier. If not, we might need to adjust our model or data. Assuming that we got good test accuracy, we're done with training and can now save the parameters $W$ and $b$ that were calculated.

In [23]:
model.save_weights('outputs/weights')

Now that our neural network has appropriate values for its parameters, we can use it to make a prediction.